In [1]:
#!/usr/bin/env python
# coding: utf-8

import argparse
import logging
import os
import sys
import time
from decimal import Decimal

import numpy as np
import pandas as pd
import scanpy as sc
import torch
from captum.attr import IntegratedGradients
from numpy.lib.function_base import gradient
from sklearn import preprocessing
from sklearn.manifold import TSNE
from sklearn.metrics import (auc, average_precision_score,
                             classification_report, mean_squared_error,
                             precision_recall_curve, r2_score, roc_auc_score)
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch import nn, optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, TensorDataset
from scipy.stats import pearsonr
import DaNN.mmd as mmd
import scanpypip.preprocessing as pp
import trainers as t
import utils as ut
from models import (AEBase, CVAEBase, DaNN, Predictor, PretrainedPredictor,
                    PretrainedVAEPredictor, TargetModel, VAEBase)
from scanpypip.utils import get_de_dataframe
from trajectory import trajectory
from sklearn.feature_selection import SelectKBest,SelectFdr
from sklearn.feature_selection import chi2

DATA_MAP={
"GSE117872":"data/GSE117872/GSE117872_good_Data_TPM.txt",
"GSE117309":'data/GSE117309/filtered_gene_bc_matrices_HBCx-22/hg19/',
"GSE117309_TAMR":'data/GSE117309/filtered_gene_bc_matrices_HBCx22-TAMR/hg19/',
"GSE121107":'data/GSE121107/GSM3426289_untreated_out_gene_exon_tagged.dge.txt',
"GSE121107_1H":'data/GSE121107/GSM3426290_entinostat_1hr_out_gene_exon_tagged.dge.txt',
"GSE121107_6H":'data/GSE121107/GSM3426291_entinostat_6hr_out_gene_exon_tagged.dge.txt',
"GSE111014":'data/GSE111014/',
"GSE110894":"data/GSE110894/GSE110894.csv",
"GSE122843":"data/GSE122843/GSE122843.txt",
"GSE112274":"data/GSE112274/GSE112274_cell_gene_FPKM.csv",
"GSE116237":"data/GSE116237/GSE116237_bulkRNAseq_expressionMatrix.txt",
"GSE108383":"data/GSE108383/GSE108383_Melanoma_fluidigm.txt",
"GSE140440":"data/GSE140440/GSE140440.csv",
"GSE129730":"data/GSE129730/GSE129730.h5ad",
"GSE149383":"data/GSE149383/erl_total_data_2K.csv",
"GSE110894_small":"data/GSE110894/GSE110894_small.h5ad"
}

In [2]:
for i in DATA_MAP:
    print(i)

GSE117872
GSE117309
GSE117309_TAMR
GSE121107
GSE121107_1H
GSE121107_6H
GSE111014
GSE110894
GSE122843
GSE112274
GSE116237
GSE108383
GSE140440
GSE129730
GSE149383
GSE110894_small


# Load arguments

In [3]:
parser = argparse.ArgumentParser()
# data 
parser.add_argument('--source_data', type=str, default='data/GDSC2_expression.csv')
parser.add_argument('--label_path', type=str, default='data/GDSC2_label_9drugs_binary.csv')
parser.add_argument('--target_data', type=str, default="GSE110894")
parser.add_argument('--drug', type=str, default='I-BET-762')
parser.add_argument('--missing_value', type=int, default=1)
parser.add_argument('--test_size', type=float, default=0.2)
parser.add_argument('--valid_size', type=float, default=0.2)
parser.add_argument('--var_genes_disp', type=float, default=0)
parser.add_argument('--min_n_genes', type=int, default=0)
parser.add_argument('--max_n_genes', type=int, default=20000)
parser.add_argument('--min_g', type=int, default=200)
parser.add_argument('--min_c', type=int, default=3)
parser.add_argument('--cluster_res', type=float, default=0.3)
parser.add_argument('--remove_genes', type=int, default=1)
parser.add_argument('--mmd_weight', type=float, default=0.25)

# train
parser.add_argument('--source_model_path','-s', type=str, default='saved/models/BET_dw_256_AE.pkl')
parser.add_argument('--target_model_path', '-p',  type=str, default='saved/models/GSE110894_I-BET-762256AE')
parser.add_argument('--pretrain', type=str, default='saved/models/GSE110894_I-BET-762_256_ae.pkl')
parser.add_argument('--transfer', type=str, default="DaNN")

parser.add_argument('--lr', type=float, default=1e-2)
parser.add_argument('--epochs', type=int, default=500)
parser.add_argument('--batch_size', type=int, default=200)
parser.add_argument('--bottleneck', type=int, default=256)
parser.add_argument('--dimreduce', type=str, default="AE")
parser.add_argument('--predictor', type=str, default="DNN")
parser.add_argument('--freeze_pretrain', type=int, default=0)
parser.add_argument('--source_h_dims', type=str, default="256,256")
parser.add_argument('--target_h_dims', type=str, default="256,256")
parser.add_argument('--p_h_dims', type=str, default="128,64")
parser.add_argument('--predition', type=str, default="classification")
parser.add_argument('--VAErepram', type=int, default=1)
parser.add_argument('--batch_id', type=str, default="HN137")
parser.add_argument('--load_target_model', type=int, default=0)
parser.add_argument('--GAMMA_mmd', type=int, default=1000)

parser.add_argument('--runs', type=int, default=1)

# Analysis
parser.add_argument('--n_DL_genes', type=int, default=50)
parser.add_argument('--n_DE_genes', type=int, default=50)


# misc
parser.add_argument('--message', '-m',  type=str, default='message')
parser.add_argument('--output_name', '-n',  type=str, default='saved/results')
parser.add_argument('--logging_file', '-l',  type=str, default='saved/logs/transfer_')

#
args, unknown = parser.parse_known_args()

In [4]:
# Read parameters
epochs = args.epochs
dim_au_out = args.bottleneck #8, 16, 32, 64, 128, 256,512
na = args.missing_value
data_path = DATA_MAP[args.target_data]
test_size = args.test_size
select_drug = args.drug
freeze = args.freeze_pretrain
valid_size = args.valid_size
g_disperson = args.var_genes_disp
min_n_genes = args.min_n_genes
max_n_genes = args.max_n_genes
source_model_path = args.source_model_path
target_model_path = args.target_model_path 
log_path = args.logging_file
batch_size = args.batch_size
encoder_hdims = args.source_h_dims.split(",")
encoder_hdims = list(map(int, encoder_hdims))
source_data_path = args.source_data 
pretrain = args.pretrain
prediction = args.predition
data_name = args.target_data
label_path = args.label_path
reduce_model = args.dimreduce
predict_hdims = args.p_h_dims.split(",")
predict_hdims = list(map(int, predict_hdims))
leiden_res = args.cluster_res
load_model = bool(args.load_target_model)

In [5]:
# Misc
now=time.strftime("%Y-%m-%d-%H-%M-%S")
# Initialize logging and std out
out_path = log_path+now+".err"
log_path = log_path+now+".log"

out=open(out_path,"w")
sys.stderr=out

#Logging infomaion
logging.basicConfig(level=logging.INFO,
                filename=log_path,
                filemode='a',
                format=
                '%(asctime)s - %(pathname)s[line:%(lineno)d] - %(levelname)s: %(message)s'
                )
logging.getLogger('matplotlib.font_manager').disabled = True


logging.info(args)

# Save arguments
#args_df = ut.save_arguments(args,now)
################################################# END SECTION OF LOADING PARAMETERS #################################################

################################################# START SECTION OF SINGLE CELL DATA REPROCESSING #################################################
#

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


In [6]:
dim0 = []
dim1 = []
avg_drop = []
data_names = []
for d in DATA_MAP:
    data_path = DATA_MAP[d]
    try:
        adata = pp.read_sc_file(data_path)
        
        sc.pp.filter_cells(adata, min_genes=200)
        sc.pp.filter_genes(adata, min_cells=3)
        adata.raw = adata
        
        
        dim0.append(adata.raw.shape[0])
        dim1.append(adata.raw.shape[1])

        sc.pp.calculate_qc_metrics(adata,inplace=True)   
        avg_drop.append(np.mean(adata.var.pct_dropout_by_counts))
        data_names.append(d)

    except:
        print("erorr in dataset ",d)
#     if data_name == 'GSE117872':
#         adata =  ut.specific_process(adata,dataname=data_name,select_origin=args.batch_id)
#     elif data_name =='GSE122843':
#         adata =  ut.specific_process(adata,dataname=data_name)
#     elif data_name =='GSE110894':
#         adata =  ut.specific_process(adata,dataname=data_name)
#     elif data_name =='GSE112274':
#         adata =  ut.specific_process(adata,dataname=data_name)
#     elif data_name =='GSE116237':
#         adata =  ut.specific_process(adata,dataname=data_name)
#     elif data_name =='GSE108383':
#         adata =  ut.specific_process(adata,dataname=data_name)
#     elif data_name =='GSE140440':
#         adata =  ut.specific_process(adata,dataname=data_name)
#     elif data_name =='GSE129730':
#         adata =  ut.specific_process(adata,dataname=data_name)
#     elif data_name =='GSE149383':
#         adata =  ut.specific_process(adata,dataname=data_name)
#     else:
#         adata=adata



erorr in dataset  GSE110894_small


In [11]:
adata

AnnData object with n_obs × n_vars = 2730 × 18380
    obs: 'n_genes', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes'
    var: 'n_cells', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'

In [7]:
data_names

['GSE117872',
 'GSE117309',
 'GSE117309_TAMR',
 'GSE121107',
 'GSE121107_1H',
 'GSE121107_6H',
 'GSE111014',
 'GSE110894',
 'GSE122843',
 'GSE112274',
 'GSE116237',
 'GSE108383',
 'GSE140440',
 'GSE129730',
 'GSE149383']

In [8]:
avg_drop

[59.336375011444424,
 85.29734034372258,
 80.26494751303305,
 93.09796119667314,
 91.40423790584785,
 92.9842869418929,
 96.3271487526065,
 73.17821206446034,
 74.72669386405016,
 71.27519862577707,
 8.96708411836419,
 43.59471239080312,
 64.22213824844935,
 92.53091791105885,
 94.26569132716831]

In [9]:
dim1

[18120,
 16130,
 15975,
 16441,
 18382,
 16456,
 19105,
 21182,
 20855,
 26981,
 12890,
 20225,
 32197,
 14725,
 18380]

In [10]:
dim0

[1302,
 1564,
 753,
 4648,
 8015,
 3658,
 42675,
 1419,
 1496,
 507,
 7,
 327,
 324,
 47382,
 2730]

In [ ]:
avg_drop

In [ ]:
np.mean(adata.var.pct_dropout_by_counts)

In [ ]:
df_stat = pd.DataFrame({"GEO":data_names,"dim0":dim0,"dim1":dim1,"avg_drop":avg_drop})

In [ ]:
df_stat

In [ ]:
# Show statisctic after QX
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt-'],
            jitter=0.4, multi_panel=True,save=data_name,show=False)
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mt-',show=False)
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts',show=False)

if args.remove_genes == 0:
    r_genes = []
else:
    r_genes = REMOVE_GENES
#Preprocess data by filtering
if data_name not in ['GSE112274','GSE140440']:
    adata = pp.receipe_my(adata,l_n_genes=min_n_genes,r_n_genes=max_n_genes,filter_mincells=args.min_c,
                        filter_mingenes=args.min_g,normalize=True,log=True,remove_genes=r_genes)
else:
    adata = pp.receipe_my(adata,l_n_genes=min_n_genes,r_n_genes=max_n_genes,filter_mincells=args.min_c,percent_mito = 100,
                        filter_mingenes=args.min_g,normalize=True,log=True,remove_genes=r_genes)